**Pattern 1** — Fixed-Size Moving Average (Count-Based Window)

**Problem:** Compute a rolling average of the last k latency values for a service.

**Task:** Given a stream/list of latencies and window size k, return the moving average for every full window.

In [0]:
from collections import deque

def moving_avg(latencies, k):
    q = deque()
    s = 0
    out = []
    for x in latencies:
        q.append(x)
        s += x
        if len(q) > k:          # drop oldest
            s -= q.popleft()
        if len(q) == k:         # window full
            out.append(s / k)
    return out

"""
Idea (Fixed-size sliding window by count)

Maintain a deque and a running sum.
Each new value enters from the right.
If window size exceeds k, remove from the left and subtract from sum.
When size == k, current average is sum / k.
"""

**Why this is Sliding Window**
Window “slides” one element at a time over the list, updating aggregate incrementally instead of recomputing from scratch.

**Time & Space Complexity**
Time: Each element is added once and removed at most once → O(n).
Space: At most k elements stored → O(k).

**Pattern 2** — Maintain Last N Events (Event Buffer)

**Problem:** Always keep the last 100 events received from a telemetry stream.

**Task:** Implement a buffer that automatically drops the oldest event once it exceeds size N.

In [0]:
from collections import deque

def make_event_buffer(max_events=100):
    return deque(maxlen=max_events)

# usage
buffer = make_event_buffer(100)
for event in incoming_events:
    buffer.append(event)   # always holds last 100

"""
Idea (Bounded sliding buffer)

Use deque(maxlen=N) so it auto-evicts the oldest item.
You don’t manually pop; Python handles it.
You can inspect, aggregate, or run heuristics on the latest N events at any time.
"""

**Why this is Sliding Window**
The window is the last N events. Each new event slides the window forward, dropping the oldest if full.

**Time & Space Complexity**
Appending each event: O(1) amortized.
Space: Bounded by N events → O(N).

**Pattern 3** — Rate Limiting (Events per Second)

**Problem:** Allow at most L requests per second from a client.

**Task:** Given a deque of recent timestamps and a per-second limit, decide if a new request is allowed.

In [0]:
from collections import deque
from time import time

def is_allowed(qps_window: deque, limit: int) -> bool:
    now = time()
    # drop timestamps older than 1 second
    while qps_window and now - qps_window[0] > 1:
        qps_window.popleft()
    if len(qps_window) < limit:
        qps_window.append(now)
        return True
    return False

# usage
qps_window = deque()
if is_allowed(qps_window, limit=100):
    handle_request()
else:
    reject_request()


"""
Idea (Time-based sliding window in seconds)

Maintain timestamps of recent requests in a deque.
Before admitting a new request, evict all timestamps older than 1 sec.
If remaining count < limit, accept and append; otherwise reject.
"""

**Why this is Sliding Window**
Window is “last 1 second”. At each call, you slide the window to [now-1s, now] and count elements inside.

**Time & Space Complexity**
Time per check: Evicting old entries happens once per event → amortized O(1).
Space: At most limit timestamps in any 1-second window → O(limit).

**Pattern 4** — Time-Based Window for Streaming Analytics

**Problem:** Maintain all events from the last 5 minutes (300s) for clickstream analytics.

**Task:** Given incoming event timestamps, keep only events in the last window_sec seconds.

In [0]:
from time import time
from collections import deque

def ingest_event(event_ts: float, window: deque, window_sec: int = 300):
    now = event_ts  # or time() for real-time
    window.append(now)
    # drop events older than window_sec
    while window and now - window[0] > window_sec:
        window.popleft()

# usage
window = deque()
for event in events_stream:
    ingest_event(event["ts"], window, window_sec=300)
    # window now contains last 5 min of events

"""
Idea (Time-based sliding window by timestamps)

Treat the deque as a time-ordered buffer.
Append new event time at the right.
Repeatedly pop from the left while the oldest event is outside [now - window_sec, now].
The deque always holds exactly the active window.
"""

**Why this is Sliding Window**
Window is defined by time, not count. It continuously moves as now advances.

**Time & Space Complexity**
Time per event: Each event is appended once and removed once → amortized O(1).
Space: Bounded by number of events that can arrive in window_sec → O(R * window_sec) where R is event rate.

**Pattern 5** — Sliding Window Maximum (Monotonic Deque)

**Problem:** For each window of size k, find the maximum element.

**Task:** Given a list of CPU utilization percentages, output the maximum CPU in every contiguous window of size k.

In [0]:
from collections import deque

def window_max(nums, k):
    q = deque()   # indices, values decreasing
    out = []

    for i, x in enumerate(nums):
        # remove smaller values from the right
        while q and nums[q[-1]] <= x:
            q.pop()

        q.append(i)

        # remove leftmost if it's outside the window
        if q[0] <= i - k:
            q.popleft()

        # window ready
        if i >= k - 1:
            out.append(nums[q[0]])
    return out

"""
Idea (Monotonic deque)

Keep indices in a deque such that their values are in decreasing order.
Front of deque = index of current window maximum.
For each new element:

Pop from right while current value ≥ last value (they can't be future maxima).

Append current index.

Pop from left if the index is outside the current window.
When window is full, max = value at q[0].
"""

**Why this is Sliding Window**
We slide a window of length k and maintain the aggregate (max) incrementally using a monotonic structure.

**Time & Space Complexity**
Time: Each index is pushed and popped at most once → O(n).
Space: Deque holds at most k indices → O(k).

**Pattern 6 **— Sliding Window Count of Errors

**Problem:** Track number of errors in the last 60 seconds to compute an error rate.

**Task:** Each time an error occurs, record it and return how many errors happened in the last window_sec seconds.

In [0]:
from collections import deque
from time import time

def record_error(error_times: deque, window_sec: int = 60) -> int:
    now = time()
    error_times.append(now)
    # drop old error timestamps
    while error_times and now - error_times[0] > window_sec:
        error_times.popleft()
    return len(error_times)

# usage
error_times = deque()
def on_error():
    count_last_minute = record_error(error_times, window_sec=60)
    # trigger alert if count_last_minute is too high

"""
Idea (Time-based sliding counter)

Store timestamps of error events.
On each new error, append its timestamp and evict any that are older than window_sec.
The deque length after cleanup is the error count in the active window.
"""

**Why this is Sliding Window**
Window is [now - window_sec, now]. Each new error slides the window forward and we recompute the count by eviction, not by re-scanning logs.

**Time & Space Complexity**
Time per error: Each timestamp inserted and removed at most once → amortized O(1).
Space: At most number of errors that can occur within window_sec → O(E * window_sec) where E is peak error rate.

**Summary**

deque = best tool for sliding windows & streaming

Must learn:

moving average

throughput

rate limiting

sliding max/min (monotonic queue)

time-based window

rolling aggregates

Staff-level tasks:

API QPS tracking

log/event anomaly detection

streaming CDC

fraud detection

sliding window over Kafka

last-N events in buffer